# Example: Learning Linear Regression Parameters using Singular Value Decomposition (SVD)
This example will familiarize students with using [Singular Value Decomposition]() to estimate the parameters in ordinary least squares and regularized least squares calculations. In particular, we'll look at estimating the parameters in single-index models for equity returns.

### Single index model of Sharpe
A single index model describes the return of a firm’s stock in terms of a firm-specific return and the overall market return. One of the simplest (yet still widely used) single index models was developed by [Sharpe (1963)](https://en.wikipedia.org/wiki/Single-index_model#:~:text=The%20single%2Dindex%20model%20(SIM,used%20in%20the%20finance%20industry.)). Let $R_{i}(t)\equiv\left(r_{i}\left(t\right) - r_{f}\right)$ 
and $R_{m}(t)\equiv\left(r_{m}\left(t\right)-r_{f}\right)$ denote the firm-specific and market **excess returns**, where $r_{f}$ denotes the risk-free rate of return.
Further, let $\epsilon_{i}\left(t\right)$ denote stationary normally distributed random noise
with mean zero and standard deviation $\sigma_{i}$. Then, the single index model of Sharpe is given by:

$$
\begin{equation*}
R_{i}\left(t\right) = \alpha_{i}+\beta_{i}\cdot{R}_{m}\left(t\right)+\epsilon_{i}
\left(t\right)\qquad{t=1,2,\dots,T}
\end{equation*}
$$

where $\alpha_{i}$ and $\beta_{i}$ are (unknown) model parameters: 
* $\alpha_{i}$ describes the firm-specific return not explained by the market; thus, $\alpha_{i}$ is the idiosyncratic return of firm $i$.
* $\beta_{i}$ has two interpretations. First, it measures the relationship between the excess return of firm $i$ and the excess return of the market. 
A large $\beta_{i}$ suggests that the market returns (or losses) are amplified for firm $i$, while a small $\beta_{i}$ suggests that the market returns (or losses) are damped for firm $i$. 
Second, it represents the relative risk of investing in a firm $i$ relative to the overall market.

## Setup
This example requires several external libraries and a function to compute the outer product. Let's download and install these packages and call our `Include.jl` file.

In [1]:
include("Include.jl");

  Activating project at `~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-10/L10c`
  No Changes to `~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-10/L10c/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-10/L10c/Manifest.toml`
    Updating registry at `~/.julia/registries/General.toml`
  No Changes to `~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-10/L10c/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-10/L10c/Manifest.toml`


## Prerequisites 
We gathered a daily open-high-low-close `dataset` for each firm in the [S&P500](https://en.wikipedia.org/wiki/S%26P_500) from `01-03-2018` until `03-14-2024`, along with data for a few exchange-traded funds and volatility products during that time. 
* We load the `orignal_dataset` by calling the `MyPortfolioDataSet()` function and then remove firms that do not have the maximum number of trading days, i.e., they are missing data. We save the cleaned data in the `dataset` variable.
* We'll also grab the list of firms in the `dataset` and sort them alphabetically. We'll store the sorted list of firms in the `my_list_of_tickers` dataset.
* Finally, we'll compute the excess return matrix `R` and the excess return for the market index, in this case treated as `SPY`.

In [2]:
dataset = Dict{String,DataFrame}();
original_dataset = MyPortfolioDataSet() |> x->x["dataset"];
maximum_number_trading_days = original_dataset["AAPL"] |> nrow;
for (ticker,data) ∈ original_dataset
    if (nrow(data) == maximum_number_trading_days)
        dataset[ticker] = data;
    end
end
my_list_of_tickers = keys(dataset) |> collect |> x->sort(x);

The single index model uses a market index, e.g., the [S&P500](https://en.wikipedia.org/wiki/S%26P_500), as the base from which we compute the return for firm $i$. Let's get the index in our dataset that corresponds to [SPDR SPY](https://www.ssga.com/us/en/intermediary/etfs/capabilities/spdr-core-equity-etfs/spy-sp-500), an ETF that tracks the [S&P500](https://en.wikipedia.org/wiki/S%26P_500). 
* We'll use the [findfirst function](https://docs.julialang.org/en/v1/base/strings/#Base.findfirst-Tuple{AbstractString,%20AbstractString}) on the `my_list_of_tickers` list to find the index that corresponds to `SPY`, we'll use this later. We'll store this value in the `idx_spy` variable.

In [3]:
idx_spy = findfirst(x->x=="SPY", my_list_of_tickers); 

Finally, we must calculate the `excess return` from the price information stored in the `dataset.` We'll do this for every firm in the `dataset` using the `μ(...)` function. We'll store the excess return data in the `R` matrix, and the excess return for `SPY` in the `Rₘ` variable:

In [4]:
risk_free_rate = 0.05;
R = μ(dataset, my_list_of_tickers, risk_free_rate = risk_free_rate) |> x-> transpose(x) |> Matrix;
Rₘ = R[idx_spy, :]; # this is the growth rate of the market, which we apoproximate by the SPY ETF

## Compute $\theta$ using pseudo-inverse and SVD approaches
Fill me in

In [17]:
λ = 0.0;
ticker_to_explore = "AMD";
idx_of_ticker = findfirst(x->x==ticker_to_explore, my_list_of_tickers);

In [6]:
θ̂_pinv = θ(R,idx_of_ticker, Rₘ, λ = λ, method = MyMatrixAlgebraLearningMethod())

2-element Vector{Float64}:
 0.31200223154724527
 1.7010715249478565

In [7]:
θ̂_svd = θ(R,idx_of_ticker, Rₘ, λ = λ, method = MySVDLearingMethod())

2-element Vector{Float64}:
 0.31200223154724493
 1.7010715249478572

#### Check: Do the two approaches give the same answer?

In [10]:
@assert θ̂_pinv ≈ θ̂_svd

## A deeper look at the SVD approach
Fill me in

In [14]:
Y = R[idx_of_ticker, :];
max_length = length(Y);
X = [ones(max_length) Rₘ];
IM = diagm(ones(2)); # we have two parameters -
(U,d,V) = svd(X);
Σ = diagm(d)

2×2 Matrix{Float64}:
 98.5355   0.0
  0.0     39.4475

In [15]:
r = rank(X)

2

In [25]:
θ̂ = zeros(2); # we have two parameters -
for i ∈ 1:r
    

    σᵢ = Σ[i,i];
    uᵢ = U[:,i];
    vᵢ = V[:,i];

    # compute the coefficient -
    cᵢ = (σᵢ*dot(transpose(uᵢ),Y))/(σᵢ^2 + λ)

    # compute the parameter update -
    θ̂ += cᵢ*vᵢ

    @show (i, θ̂)
end

(i, θ̂) = (1, [0.017885718683099534, 1.7041583772494766])
(i, θ̂) = (2, [0.312002231547245, 1.7010715249478539])
